Download and extract data

In [16]:
import urllib.request
import tarfile
import os

if os.path.isdir('MovieSummaries') == False:
    print("Downloading and extracting data...")
    data_tar_link = "http://www.cs.cmu.edu/~ark/personas/data/MovieSummaries.tar.gz"
    ftpstream = urllib.request.urlopen(data_tar_link)

    data_tar_file = tarfile.open(fileobj=ftpstream, mode="r|gz")
    data_tar_file.extractall()
    
    print("Dataset downloaded and extracted.")
else:
    print("Dataset already downloaded")

print("Data size: ", sum(os.path.getsize(os.path.join('MovieSummaries', f)) / (1024.0*1024.0)
          for f in os.listdir('MovieSummaries')
          if os.path.isfile(os.path.join('MovieSummaries', f))), "MB")
# this should take less than two minutes (depends mostly on connection speed), if it takes longer 
# check if MovieSummaries directory is created in working directory and some files are in it

Dataset already downloaded
Data size:  127.5914134979248 MB


In [17]:
movie_summaries = []

with open('MovieSummaries/plot_summaries.txt') as movie_summaries_file:
    for line in movie_summaries_file.readlines():
        splitted_line = line.split('\t', 1)
        movie_summaries.append((int(splitted_line[0]), splitted_line[1]))
        
print(len(movie_summaries))

42306


Ima manje opisa filmova nego filmova u movie.metadata.tsv ?

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

movie_metadata = pd.read_pickle('movie_metadata.pkl')

In [19]:
movie_metadata.head()

,wikiID,name,standard_genre
0,975900,Ghosts of Mars,"{Adventure, Horror, Science Fiction, Thriller,..."
1,3196793,Getting Away with Murder: The JonBenét Ramsey ...,"{Drama, Mystery, Biographical, Crime}"
2,28463795,Brun bitter,"{Drama, Fiction, Crime}"
3,9363483,White Of The Eye,"{Psychological, Thriller}"
4,261236,A Woman in Flames,{Drama}


In [20]:
movie_metadata['summary'] = ""

In [21]:
movie_metadata = movie_metadata.set_index('wikiID')
movie_metadata.head()

,name,standard_genre,summary
wikiID,,,
975900,Ghosts of Mars,"{Adventure, Horror, Science Fiction, Thriller,...",
3196793,Getting Away with Murder: The JonBenét Ramsey ...,"{Drama, Mystery, Biographical, Crime}",
28463795,Brun bitter,"{Drama, Fiction, Crime}",
9363483,White Of The Eye,"{Psychological, Thriller}",
261236,A Woman in Flames,{Drama},


In [22]:
cnt = 0

for movie_id, summary in movie_summaries:
    if movie_id not in movie_metadata.index:
         continue
    
    cnt += 1
    movie_metadata.at[movie_id, 'summary'] = summary
    
print(cnt)

40188


In [26]:
print(len(movie_metadata[movie_metadata['summary'] == ""]))
movie_metadata[movie_metadata['summary'] == ""].head()

34195


,name,standard_genre,summary
wikiID,,,
3196793,Getting Away with Murder: The JonBenét Ramsey ...,"{Drama, Mystery, Biographical, Crime}",
28463795,Brun bitter,"{Drama, Fiction, Crime}",
13696889,The Gangsters,{Comedy},
10408933,Alexander's Ragtime Band,"{Musical, Comedy}",
9997961,Contigo y aquí,"{Musical, Drama, Comedy}",


U README.txt pise da ukupno ima 42 306 sazetaka plotova u ovom datasetu, dakle odbacili smo oko 2000 filmova.

In [28]:
movie_metadata = movie_metadata[movie_metadata['summary'] != ""]
print(len(movie_metadata))

40188


In [29]:
movie_metadata.head()

,name,standard_genre,summary
wikiID,,,
975900,Ghosts of Mars,"{Adventure, Horror, Science Fiction, Thriller,...","Set in the second half of the 22nd century, th..."
9363483,White Of The Eye,"{Psychological, Thriller}",A series of murders of rich young women throug...
261236,A Woman in Flames,{Drama},"Eva, an upper class housewife, becomes frustra..."
18998739,The Sorcerer's Apprentice,"{Adventure, Family, Fantasy}","Every hundred years, the evil Morgana returns..."
6631279,Little city,"{Drama, Romance Film, Romance, Comedy}","Adam, a San Francisco-based artist who works a..."


In [43]:
for summary in movie_metadata['summary'][:5]:
    print(summary)

Set in the second half of the 22nd century, the film depicts Mars as a planet that has been 84% terraformed, allowing humans to walk on the surface without wearing pressure suits. The Martian society has become largely matriarchal, with women in most positions of authority. The story concerns a police officer, Melanie Ballard , second in command of a small team alongside Sergeant Jericho  sent to pick up and transport a prisoner named Desolation Williams . Arriving at the remote mining town where Williams is being held, Ballard finds virtually all of the people missing. She learns that the miners had discovered an underground doorway created by an ancient Martian civilization. When the door was opened it released "ghosts," disembodied spirits which possessed the miners. Violence ensues, as the possessed miners commit horrific acts of death and destruction, as well as self-mutilation. With their team leader Helena Bradock  murdered, Ballard must fight off the attacking miners, escape th